## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import set_config

import functions_variables as fv
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
set_config(display="diagram")
X = pd.DataFrame()
folder_name = '../data'



In [2]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


## Hyperparam Tuning

In [3]:
# perform tuning and cross validation here 
# using GridsearchCV/ RandomsearchCV (MVP)
# or your custom functions

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [4]:
# save your best model here

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [18]:
class DataLoader(TransformerMixin):
    def __init__(self, folder_name):
        self.folder_name = folder_name

    def fit(self, X=None, y=None):  
        return self  

    def transform(self, X=None): 
        data = fv.load_data(self.folder_name)  
        return data

class DataCleaner(TransformerMixin):

    def fit(self, X=None, y=None):
        return self

    def transform(self, X):  
        cleaned_data = fv.clean_data(X)  
        return cleaned_data
    
# class Scalar(TransformerMixin): <---- NEEDS FIXING
#     def fit(self, X):
#         return self
    
#     def transform(self, X):
#         numeric_columns = X.select_dtypes(include=['int']).columns + X.select_dtypes(include=['float']).columns
#         scalar = StandardScaler()
#         X = scalar.fit_transform(X[numeric_columns])
#         return X
    
class PCA_bool(TransformerMixin):
    def __init__(self, n_components):
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components)
        self.bool_columns = None

    def fit(self, X, y=None):
        self.bool_columns = X.select_dtypes(include=['bool']).columns
        self.pca.fit(X[self.bool_columns])
        return self
    
    def transform(self, X,y=None):
        X = X[self.bool_columns].copy()
        X = self.pca.transform(X)
        return X    

In [21]:
# Build pipeline here
X = pd.DataFrame()
folder_name = '../data'
pipe_load_clean = Pipeline([
    ('loader', DataLoader(folder_name)),
    ('cleaner', DataCleaner())
    
])
#scaling the numeric data
numeric_pipe = Pipeline([
    ('scaling', Scalar())
])
#scaling the boolean data
bool_pipe = Pipeline([
    ('pca', PCA_bool(n_components = 10))
])

union_pipe = FeatureUnion(transformer_list=[
    ('numeric', numeric_pipe()),
    ('boolean', PCA_bool(n_componenets = 10))
])

In [23]:
cleaned_data = pipe_load_clean.transform(X)

Not a Json: .gitkeep


c:\Users\finch\Documents\Lighthouse Labs\Midterm project\DS-Midterm-Project\notebooks\functions_variables.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, small_df], ignore_index = True)


Not a Json: license.txt
Not a Json: processed
Not a Json: uscities.csv


c:\Users\finch\Documents\Lighthouse Labs\Midterm project\DS-Midterm-Project\notebooks\functions_variables.py:215: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['garage'] = df['garage'].fillna(0)
c:\Users\finch\Documents\Lighthouse Labs\Midterm project\DS-Midterm-Project\notebooks\functions_variables.py:234: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['year_built'] = df['year_built'].fillna(mean_year)
c:\Users\finch\Documents\Lighthouse Labs\Midterm project\DS-Midterm-Project\notebooks\functions_variables.py:237: FutureWarning: Downcasting object 

Geocoding attempt 1 of 4...
Error geocoding Honolulu, Hawaii: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Honolulu%2C+Hawaii%2C+downtown&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding Atlanta, Georgia: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Atlanta%2C+Georgia%2C+downtown&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding attempt 2 of 4...


Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [30]:
numeric_columns = list(cleaned_data.select_dtypes(include=['int']).columns) + list(cleaned_data.select_dtypes(include=['float']).columns)
numeric_columns

['year_built',
 'sold_price',
 'lot_sqft',
 'sqft',
 'baths',
 'garage',
 'stories',
 'beds',
 'postal_code',
 'tags_basement',
 'tags_basketball',
 'tags_basketball_court',
 'tags_beach',
 'tags_beautiful_backyard',
 'tags_big_bathroom',
 'tags_big_lot',
 'tags_big_yard',
 'tags_carport',
 'tags_cathedral_ceiling',
 'tags_central_air',
 'tags_central_heat',
 'tags_city_view',
 'tags_clubhouse',
 'tags_coffer_ceiling',
 'tags_community_boat_facilities',
 'tags_community_center',
 'tags_community_clubhouse',
 'tags_community_golf',
 'tags_community_gym',
 'tags_community_outdoor_space',
 'tags_community_park',
 'tags_community_security_features',
 'tags_community_spa_or_hot_tub',
 'tags_community_swimming_pool',
 'tags_community_tennis_court',
 'tags_corner_lot',
 'tags_cul_de_sac',
 'tags_den_or_office',
 'tags_dining_room',
 'tags_disability_features',
 'tags_dishwasher',
 'tags_efficient',
 'tags_elevator',
 'tags_energy_efficient',
 'tags_ensuite',
 'tags_exposed_brick',
 'tags_fami

,tags_basement,tags_basketball,tags_basketball_court,tags_beach,tags_beautiful_backyard,tags_big_bathroom,tags_big_lot,tags_big_yard,tags_carport,tags_cathedral_ceiling,...,tags_two_or_more_stories,tags_updated_kitchen,tags_vaulted_ceiling,tags_view,tags_views,tags_washer_dryer,tags_water_view,tags_waterfront,tags_wooded_land,tags_nan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1469,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1470,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# save your pipeline here
cleaned_data

,property_id,year_built,sold_price,lot_sqft,sqft,baths,garage,stories,beds,type,...,tags_vaulted_ceiling,tags_view,tags_views,tags_washer_dryer,tags_water_view,tags_waterfront,tags_wooded_land,tags_nan,city_lat,city_lon
0,8846541030,1998,129900,11761,1478,2,2,1,3,single_family,...,0,0,0,0,0,0,0,0,32.374725,-86.312465
1,7727981021,1945,88500,6534,1389,2,1,2,4,single_family,...,0,0,0,0,0,0,0,0,32.374725,-86.312465
2,7320925131,1969,145000,17424,2058,2,0,1,3,single_family,...,0,0,0,0,0,0,0,0,32.374725,-86.312465
3,7231604965,1955,65000,9712,1432,2,0,1,3,single_family,...,0,0,0,0,0,0,0,0,32.374725,-86.312465
4,7700690979,1984,169000,10890,1804,2,0,1,3,single_family,...,0,0,0,0,0,0,0,0,32.374725,-86.312465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,4542127284,1910,99000,4792,1214,1,1,2,3,single_family,...,0,0,0,0,0,0,0,0,38.356629,-81.644059
1469,3895826397,1860,29700,7841,988,1,0,1,3,single_family,...,0,0,0,0,0,0,0,0,38.356629,-81.644059
1470,4941005485,1860,162250,65340,1470,1,0,1,3,single_family,...,0,0,0,0,0,0,0,0,38.356629,-81.644059
1471,4306867390,1860,63800,17428,1526,0,0,1,0,single_family,...,0,0,0,0,0,0,0,0,38.356629,-81.644059
